## Define the Objective of the Analysis

...

Analisi del sentiment relativo al dibattito sullo _Smart Working_ in Italia e individuazione di eventuali 'influencer' sul tema.

## Collect Data

In [1]:
# imports
import numpy as np
import pandas as pd
import tweepy
tweepy.__version__

import re
import string
import itertools
from collections import Counter
from datetime import datetime
from numpy.core.multiarray import result_type
import time

In [2]:
# insert the keys here
consumer_key = 'VPzjkqKl2y1uSTJQvnVqS9e1X' 
consumer_secret = 'STG2IzVMf65vPGeOvBQyzdeoKBExAr5sIkhOaBeDe2fnIN14vY'
access_token = '1508409949835214853-HIyZJ3oT32TijKsdNDhGFZEEQTWwau'
access_token_secret = 'uLcs9hUYmLdocxkaSfXo69Gii46TISu5qZj5F6f6fBfnW'

#### Tweets Download

The next step is creating an OAuthHandler instance. We pass our consumer key and access token which we defined above.

In [3]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

Next, we pass the OAuthHandler instance into the API method.

In [4]:
api = tweepy.API(auth, wait_on_rate_limit=True)

Tweets that contain a specific keyword

In [5]:
from tqdm.notebook import tqdm

import time
keywords = '("smartworking" OR "remotework" OR "lavoroagile" OR "hybridwork" OR "smart working" OR "remote work" OR "lavoro agile" OR "hybrid work")'

list_tweets = []

for tweet in tweepy.Cursor(api.search_tweets, q=keywords, count=100, lang='it').items(10000):
  print('entering')
  full_text = api.get_status(tweet.id, tweet_mode='extended')._json['full_text']
  print(tweet.id)
  list_tweets.append([tweet.created_at, tweet.id, full_text, tweet.favorite_count, tweet.retweet_count, tweet.user.screen_name,
                      tweet.user.location, tweet.retweeted, tweet.entities['user_mentions'], tweet.entities['hashtags']])

# items is the maximum number of tweets to download.
# count is the number of tweets to return per page, up to a maximum of 100.

entering
1610349101027049474
entering
1610347094627205120
entering
1610346649888632832
entering
1610343335352213504
entering
1610342736166621184
entering
1610340031218040832
entering
1610339854944206848
entering
1610334452013563906
entering
1610333781704974336
entering
1610330919797420032
entering
1610330688691085315
entering
1610329843077349384
entering
1610326421615910912
entering
1610325202017058817
entering
1610320603139145731
entering
1610319949666336769
entering
1610315013239050240
entering
1610313126875914241
entering
1610312511710011393
entering
1610311174788096001
entering
1610310279321227265
entering
1610309938881957893
entering
1610309082233044997
entering
1610307509167984641
entering
1610306243507400705
entering
1610305083425673228
entering
1610303676098416647
entering
1610301347206774784
entering
1610301230852591617
entering
1610296424448540673
entering
1610296208240640001
entering
1610295092882935809
entering
1610293965487763458
entering
1610292835374219266
entering
16102

Rate limit reached. Sleeping for: 640


entering
1608866524328259586
entering
1608866050493534209
entering
1608865894922338304
entering
1608865680635621378
entering
1608865420555227145
entering
1608865373704843265
entering
1608865332437069825
entering
1608864904378986497
entering
1608864660509589504
entering
1608864292266446850
entering
1608863784810188800
entering
1608863373550313477
entering
1608863338884366342
entering
1608863017948807174
entering
1608862848381497345
entering
1608862499134398464
entering
1608862459326271488
entering
1608862032438394882
entering
1608861904000397312
entering
1608861773901492224
entering
1608861633530445824
entering
1608861405586808835
entering
1608861338641772545
entering
1608860548950630402
entering
1608860390917607424
entering
1608860007155761153
entering
1608859743279353856
entering
1608859455822696449
entering
1608859277338488835
entering
1608858729323335684
entering
1608858463941296134
entering
1608858439396241411
entering
1608857792668119040
entering
1608857064880869378
entering
16088

Rate limit reached. Sleeping for: 668


entering
1607894958547034114
entering
1607892768277532672
entering
1607890979977043971
entering
1607890168945541127
entering
1607889759606652928
entering
1607888531006816257
entering
1607887927975059456
entering
1607884169111126016
entering
1607883127799828481
entering
1607882463279480833
entering
1607880447853481984
entering
1607880287135875072
entering
1607879903617097728
entering
1607878707498176517
entering
1607878629584961536
entering
1607876230325452800
entering
1607875580103462913
entering
1607874993765101569
entering
1607874795672338432
entering
1607874079100473345
entering
1607872137833979906
entering
1607871728918708225
entering
1607867308629979137
entering
1607866125651509249
entering
1607864309933301760
entering
1607863219107893250
entering
1607862187648815105
entering
1607859436051128322
entering
1607859079526981633
entering
1607858343317569536
entering
1607857909874200578
entering
1607857560672944129
entering
1607857328459583490
entering
1607856348322234368
entering
16078

In [6]:
print(len(list_tweets))

2416


In [7]:
# Turn list_tweet into a DataFrame changing column names
tweets = pd.DataFrame(list_tweets, columns=['date','id','text','like','n_rt','author','location','retweeted','user_mentions','hastags'])
tweets.to_csv('../data/SW_ex.csv')

In [8]:
# Read csv
#tweets = pd.read_csv('../data/SW_ex.csv')
#tweets_loaded = tweets_loaded.drop('Unnamed: 0', axis=1)
tweets.head()

,date,id,text,like,n_rt,author,location,retweeted,user_mentions,hastags
0,2023-01-03 18:55:19+00:00,1610349101027049474,#smartworking senza vincoli inutili ovunque po...,0,0,jymbo79,,False,[],"[{'text': 'smartworking', 'indices': [0, 13]}]"
1,2023-01-03 18:47:20+00:00,1610347094627205120,"🔸 #Smartworking in vetta, #ValGardena al terzo...",1,0,sole24ore,,False,[],"[{'text': 'Smartworking', 'indices': [2, 15]},..."
2,2023-01-03 18:45:34+00:00,1610346649888632832,"Smartworking in vetta, Val Gardena al terzo po...",0,0,amcaserodano,"Rodano, piazza del Funtanin 1",False,[],[]
3,2023-01-03 18:32:24+00:00,1610343335352213504,"RT @bioccolo: #Mattarella ha detto: ""Pensare d...",0,5,BerrettiniC,,False,"[{'screen_name': 'bioccolo', 'name': 'bioccolo...","[{'text': 'Mattarella', 'indices': [14, 25]}]"
4,2023-01-03 18:30:01+00:00,1610342736166621184,Da smart working a Rdc: come cambia il mondo d...,0,0,bizcommunityit,,False,[],[]


In [9]:
print(tweets.shape)
print(tweets.columns)

(2416, 10)
Index(['date', 'id', 'text', 'like', 'n_rt', 'author', 'location', 'retweeted',
       'user_mentions', 'hastags'],
      dtype='object')


#### Data Pre-Processing

In [10]:
# Remove duplicates
tweets.drop_duplicates(subset ="id", inplace = True)
tweets.reset_index(drop = True, inplace = True)
tweets.shape

(2416, 10)

In [12]:
# Change date format
day = tweets['date'].dt.day
month = tweets['date'].dt.month
year = tweets['date'].dt.year

date = year.astype(str) + month.astype(str).str.zfill(2) + day.astype(str).str.zfill(2)
date = pd.to_datetime(date, format='%Y%m%d')
tweets.drop(columns = ['date'], inplace = True)
tweets['date'] = date

# Reorder columns
cols = tweets.columns.tolist()
cols = cols[-1:] + cols[:-1]
tweets = tweets[cols].copy()

print('Tweet per day:')
print()
print(tweets.groupby('date').count()['id'])

Tweet per day:

date
2022-12-24     32
2022-12-25     83
2022-12-26    109
2022-12-27    387
2022-12-28    329
2022-12-29    200
2022-12-30    562
2022-12-31    276
2023-01-01    145
2023-01-02    158
2023-01-03    135
Name: id, dtype: int64


In [13]:
# Create a DataFrame with the authors of the tweets and their respective frequency
freq_authors = tweets['author'].value_counts()
freq_authors.head()

mrmcphisto         67
infoitinterno      45
Lucapadova25       33
lucague46760617    29
aleale2780         27
Name: author, dtype: int64

In [14]:
# Extract all hashtags from the full text
tweets['hashtags_list'] = tweets['text'].apply(lambda x: re.findall(r"#(\w+)", x))

# Extract all mentions from the full text
tweets['mentions'] = tweets['text'].apply(lambda x: re.findall(r"@(\w+)", x))

# \w matches any single letter, number or underscore (same as [a zA Z0 9_])

#### Data Cleaning

NLTK - Natural Language ToolKit is a platform for building Python programs to work with human language data. It provides easy to use interfaces to over 50 corpora and lexical resources, along with a suite of text processing libraries for classification, tokenization, stemming, tagging, parsing, and semantic reasoning.

In [15]:
import nltk
from nltk import FreqDist
nltk.download
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer

from wordcloud import WordCloud
import matplotlib.pyplot as plt
import seaborn as sns

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\FC\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\FC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\FC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\FC\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Text Pre-Processing

In [16]:
def preprocess_data(data):
    # remove numbers and turning words into lower case
    data = data.astype(str).str.replace('\d+','')
    lower_text = data.str.lower()
    lemmatizer = nltk.stem.WordNetLemmatizer()
    w_tokenizer = TweetTokenizer()
    
    # token lemmatization (ex. goes --> go)
    def lemmatize_text(text):
        return[(lemmatizer.lemmatize(w)) for w in w_tokenizer.tokenize((text))]
    
    # remove punctuation
    def remove_punctuation(words):
        new_words = []
        for word in words:
            new_word = re.sub(r'[\w\s]', '', (word))
            if new_word != '':
                new_words.append(new_word)
        return new_words
    
    words = lower_text.apply(lemmatize_text)
    words = words.apply(remove_punctuation)
    return pd.DataFrame(words)

In [17]:
# apply preprocess_data function
pre_tweets = preprocess_data(tweets['text'])
tweets['text_proc'] = pre_tweets

# delete italian stopwords
stop_words = set(stopwords.words('italian'))
tweets['text_proc'] = tweets['text_proc'].apply(lambda x: [item for item in x if item not in stop_words])

C:\Users\FC\AppData\Local\Temp\ipykernel_8208\3179111472.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  data = data.astype(str).str.replace('\d+','')


## Analysis

### Social Content Analysis

#### _Sentiment Analysis_

FEEL-IT: Emotion and Sentiment Classification for the Italian Language.

https://towardsdatascience.com/sentiment-analysis-and-emotion-recognition-in-italian-using-bert-92f5c8fe8a2

In [18]:
from feel_it import EmotionClassifier, SentimentClassifier

sentiment_classifier = SentimentClassifier()
emotion_classifier = EmotionClassifier()

c:\Users\FC\miniconda3\envs\twitter\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The feel-it-italian-sentiment model performs sentiment analysis on Italian. We fine-tuned the UmBERTo model on our new dataset (i.e., FEEL-IT) obtaining state-of-the-art performances on different benchmark corpora.

In [ ]:
print(sentiment_classifier.predict(tweets["text"].values.tolist()))

In [21]:
tweets_sentiment = tweets.copy()

In [22]:
text = tweets_sentiment['text']
li_sent = []
for i in range(0, text.shape[0]):
  sent = sentiment_classifier.predict([text[i]])
  li_sent.append(sent)

tweets_sentiment['sentiment_BERT'] = [item for sublist in li_sent for item in sublist]
     
positive = []
negative = []

Riga 0 su 2416


In [23]:
tweets_sentiment.columns

Index(['date', 'id', 'text', 'like', 'n_rt', 'author', 'location', 'retweeted',
       'user_mentions', 'hastags', 'hashtags_list', 'mentions', 'text_proc',
       'sentiment_BERT'],
      dtype='object')

In [24]:
positive = []
negative = []
ratio = []

for line in tweets_sentiment.values:
  sent = line[13]

  if sent == 'negative':
    positive.append(0)
    negative.append(1)
    ratio.append(-1)
  else:
    positive.append(1)
    negative.append(0)
    ratio.append(1)

In [25]:
tweets_sentiment['positive'] = positive
tweets_sentiment['negative'] = negative
tweets_sentiment['ratio'] = ratio

In [26]:
tweets_sentiment.to_csv('../data/SW_sentiment_ex.csv')

#### _Emotion Analysis_

Recognizing emotions in text is fundamental to get a better sense of how people are talking about something. People can talk about a new event, but positive/negative labels might not be enough.

The feel-it-italian-emotion model performs emotion classification (joy, fear, anger, sadness) on Italian. We fine-tuned the UmBERTo model on our new dataset (i.e., FEEL-IT) obtaining state-of-the-art performances on different benchmark corpora.

In [27]:
tweets_emotion = tweets.copy()

In [28]:
text = tweets_emotion['text']
li_emotion = []
for i in range(0, text.shape[0]):
  emotion = emotion_classifier.predict([text[i]])
  li_emotion.append(emotion)

tweets_emotion['emotion_BERT'] = [item for sublist in li_emotion for item in sublist]
     
anger = []
joy = []
fear = []
sadness = []

In [29]:
anger = []
joy = []
fear = []
sadness = []

for line in tweets_emotion.values:
  emotion = line[13]

  if emotion == 'anger':
    anger.append(1)
    joy.append(0)
    fear.append(0)
    sadness.append(0)
  elif emotion == 'joy':
    anger.append(0)
    joy.append(1)
    fear.append(0)
    sadness.append(0)
  elif emotion == 'fear':
    anger.append(0)
    joy.append(0)
    fear.append(1)
    sadness.append(0)
  else:
    anger.append(0)
    joy.append(0)
    fear.append(0)
    sadness.append(1)

In [30]:
tweets_emotion['anger'] = anger
tweets_emotion['joy'] = joy
tweets_emotion['fear'] = fear
tweets_emotion['sadness'] = sadness

In [31]:
tweets_emotion.to_csv('../data/SW_emotion_ex.csv')

---

#### Social Network Analysis

Measures of Centrality

Community Detection

## Visualization